In [1]:
import pyspark
from delta import *
from pyspark.sql.functions import lit

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.network.timeout', '800s') \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


24/02/17 17:26:22 WARN Utils: Your hostname, Henry-Desktop resolves to a loopback address: 127.0.1.1; using 172.20.208.84 instead (on interface eth0)
24/02/17 17:26:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/henry/anaconda3/envs/BCI/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/henry/.ivy2/cache
The jars for the packages stored in: /home/henry/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-64ce047f-71df-42a6-9b36-3ff681eef771;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 76ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

24/02/17 17:26:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
pathGlobFilter = '*.csv'

takeoff = spark.read.options(delimiter=',').csv(
    "data/takeoff", inferSchema=True, pathGlobFilter=pathGlobFilter, header=True)
landdf = spark.read.options(delimiter=',').csv(
    "data/land", inferSchema=True, pathGlobFilter=pathGlobFilter, header=True)
leftdf = spark.read.options(delimiter=',').csv(
    "data/left", inferSchema=True, pathGlobFilter=pathGlobFilter, header=True)
rightdf = spark.read.options(delimiter=',').csv(
    "data/right", inferSchema=True, pathGlobFilter=pathGlobFilter, header=True)
forward = spark.read.options(delimiter=',').csv(
    "data/forward", inferSchema=True, pathGlobFilter=pathGlobFilter, header=True)
backward = spark.read.options(delimiter=',').csv(
    "data/backward", inferSchema=True, pathGlobFilter=pathGlobFilter, header=True)

In [13]:
takeoff.dtypes

[('Sample Index', 'double'),
 (' EXG Channel 0', 'double'),
 (' EXG Channel 1', 'double'),
 (' EXG Channel 2', 'double'),
 (' EXG Channel 3', 'double'),
 (' EXG Channel 4', 'double'),
 (' EXG Channel 5', 'double'),
 (' EXG Channel 6', 'double'),
 (' EXG Channel 7', 'double'),
 (' EXG Channel 8', 'double'),
 (' EXG Channel 9', 'double'),
 (' EXG Channel 10', 'double'),
 (' EXG Channel 11', 'double'),
 (' EXG Channel 12', 'double'),
 (' EXG Channel 13', 'double'),
 (' EXG Channel 14', 'double'),
 (' EXG Channel 15', 'double'),
 (' Accel Channel 0', 'double'),
 (' Accel Channel 1', 'double'),
 (' Accel Channel 2', 'double'),
 (' Other', 'double'),
 (' Other.1', 'double'),
 (' Other.2', 'double'),
 (' Other.3', 'double'),
 (' Other.4', 'double'),
 (' Other.5', 'double'),
 (' Other.6', 'double'),
 (' Analog Channel 0', 'double'),
 (' Analog Channel 1', 'double'),
 (' Analog Channel 2', 'double'),
 (' Timestamp', 'double'),
 (' Other.7', 'double'),
 (' Timestamp (Formatted)', 'timestamp')]

In [11]:
takeoff.head(2)

[Row(Sample Index='248.0',  EXG Channel 0='25668.56451851899',  EXG Channel 1='12112.142099397432',  EXG Channel 2='-35910.67026980761',  EXG Channel 3='-62853.4853879792',  EXG Channel 4='-187500.02235174447',  EXG Channel 5='9056.591577123592',  EXG Channel 6='-187500.02235174447',  EXG Channel 7='-187500.02235174447',  EXG Channel 8='-37559.67021699789',  EXG Channel 9='6519.512119235053',  EXG Channel 10='-7719.420816829303',  EXG Channel 11='-70088.20862629516',  EXG Channel 12='5340.949337595623',  EXG Channel 13='-19259.648830848797',  EXG Channel 14='-5931.728295293844',  EXG Channel 15='4260.0189757369735',  Accel Channel 0='0.0',  Accel Channel 1='0.0',  Accel Channel 2='0.0',  Other='192.0',  Other.1='0.0',  Other.2='0.0',  Other.3='0.0',  Other.4='0.0',  Other.5='0.0',  Other.6='0.0',  Analog Channel 0='0.0',  Analog Channel 1='0.0',  Analog Channel 2='0.0',  Timestamp='1695418029.478212',  Other.7='0.0',  Timestamp (Formatted)=' 2023-09-22 16:27:09.478'),
 Row(Sample Index